# Data Warehouse on Azure

##  1. Azure Synapse Workspace
1. Vào **Create a resource** -> **Analytics** -> **Azure Synapse Analytics**. Chọn **Resource group** có sẵn. \
   Thêm **Workspace name**, **Account name** và **File system name**. Còn lại để nguyên -> **Next**

   <img src="images/azure1.png" width=700>

2. Điền **SQL Password**, còn lại để nguyên -> **Next** cho đến **Review + create** -> **Create**

   <img src="images/azure2.png" width=700>

3. Vào workspace vừa tạo -> **Open Synapse Studio**

   <img src="images/azure3.png" width=1500>

## 2. Azure PostgreSQl
1. Vào **Create a resource** -> **Databases** -> **Azure Database for PostgreSQL**. \
   Ấn **Configure server** -> Chọn Compute rẻ nhất -> Thiết lập Authenication, còn lại để nguyên -> **Review + create** -> **Create**

   <img src="images/azure4.png" width=800>

2. Vào Server vừa tạo -> **Settings** -> **Networking** -> Tick "Allow public access from any azure ..." \
   Trong Firewall rules, ấn "+ Add 0.0.0.0 - 255.255.255.255" -> **Save**

   <img src="images/azure5.png" width=1500>

3. Tải data: https://video.udacity-data.com/topher/2022/March/622a5fc6_azure-data-warehouse-projectdatafiles/azure-data-warehouse-projectdatafiles.zip \
   Extract ra -> Copy các files vào folder `data` (Đây cũng là data ở Course 3 Project)

4. Mở file `ProjectDataToPostgres.py` -> Điền host, user, password -> Chạy file này

5. Dùng 1 DBMS Tool (VD DataGrip, pgAdmin4) để kiểm tra các tables trong DB.

## 3. Ingesting Data into Azure Synapse
1. Vào **Synapse Studio** -> **Manage** -> **Linked services** -> **New** -> **Azure** -> **Azure Database for PostgreSQL** -> **Continue**

   <img src="images/azure6.png" width=1500>

2. Chọn **Azure subscription** có sẵn, chọn **Server name** đã tạo, chọn **Database name** là `udacityproject`, điền **User name** và **Password**. \
   Chọn **Encrytion method** là `RequestSSL`, chọn **Not validate server certificate** -> **Test conection** -> **Create**

   <img src="images/azure7.png" width=1500>

3. Tạo một Linked service nữa tới **Azure Blob Storage**. Chọn **Azure subscription** và **Storage account name** có sẵn -> **Test connection** -> **Create**.

   <img src="images/azure8.png" width=1500>

4. Ta muốn ingest data từ `AzurePostgreSql1` vào `AzureBlobStorage1`. Vào **Home** -> **Ingest** \
   Trong **Properties**, chọn **Run once now** -> **Next** \
   Trong **Source**, chọn **Source type** là **Azure Database for PostgreSQL**, **Connection** là link vừa tạo. Tick `public.payment` -> **Next**

   <img src="images/azure9.png" width=1000>

5. Có thể xem **Preview data** -> **Next** \
   Trong **Destination**, chọn **Destination type** là **Azure Blob Storage**, **Connection là link** vừa tạo. Chọn **Folder path** và điền **File name** là`publicpaymentimport.csv` -> **Next**

   <img src="images/azure10.png" width=800>

6. Trong **File format settings**, để nguyên ->  **Next** cho đến **Review and finish** -> **Finish** \
   Vào **Data** -> **Linked** -> **Azure Blob Storage** để kiểm tra data đã ingest (nếu chưa thấy thì F5).

   <img src="images/azure11.png" width=1500>

## 3. Creating Staging Tables using Azure Synapse
0. (Optional) Vào Synapse Workspace -> Security -> Networking -> Tick "Allow Azure services and resources to access this workspace" \
   Vào **Synapse Studio** -> **Manage** -> **SQL pools** -> **New** để tạo 1 Dedicated SQL pool, có thể dùng thay cho Built-in pool ở các bước sau. \
   Bắt buộc làm bước này nếu muốn tạo Linked Service từ ADF tới Synapse DB (Ở C5L2).

1. Vào **Synapse Studio** -> **Data** -> **Linked** -> **Azure Data Lake Storage Gen2** -> Chọn file như dưới.

   <img src="images/azure12.png" width=1500>

2. Ấn **New SQL script** -> **Create external table** -> Để nguyên -> **Continue** -> Chọn **SQL pool**, **database**, đặt **External table name** là `staging_payment` -> **Open script** \
   Sửa code phần `CREATE EXTERNAL TABLE` là:
   ```sql
   CREATE EXTERNAL TABLE dbo.staging_payment (
       [PaymentId] bigint,
       [PaymentDate] varchar(50),
       [Amount] float,
       [AccountNumber] bigint
       )
       WITH (
       LOCATION = 'publicpaymentimport.csv',
       DATA_SOURCE = [hieufilesystem43_hieuaccount43_dfs_core_windows_net],
       FILE_FORMAT = [SynapseDelimitedTextFormat]
       )
   GO
   ```
   Ấn **Run**

   <img src="images/azure13.png" width=1500>

   (Chú ý: Nêu chạy có error và muốn chạy lại thì trước tiên phải drop external table đã tạo. Ấn vào dấu $\vdots$ cạnh table -> **New SQL script** -> **DROP** -> **Run**)

3. Kiểm tra KQ trong **Data** -> **Workspace** -> **SQL database**

   <img src="images/azure14.png" width=400>